# (2) PREPARING THE DATA FOR PYTORCH NEURAL NETWORKS


**Neural network in pyTorch is based on tensors.Tensors are similar to numpy arrays but they can run on graphic processing unit and tensor processing unit where as numpy arrays only run on central processing unit.Hence, working with tensors greatly reduces the training time.
Machine learning can be performed on any data as long as we are able to convert them into tensors.**

Working flow-><br>
1) Merging all the csv(s) in defog and tdcsfog training folder.<br>
2) To create a single main dataframe.<br>
3) Balancing the main dataframe.<br>
4) splitting the main dataframe into three dataframes for three models.<br>
5) Making three pytorch neural network models for starthesitation,turn and walk.<br>
6) Training the models.<br>
7) Evaluating the model based on loss and accuracy.<br>
8) Predicting on test dataframe.<br>***

In [2]:
#loading the kaggle dataset
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessiona

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/sample_submission.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/daily_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/48b636e0f5.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/9fb7805d99.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/e658b0aa3d.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/1c3719ea59.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/164adaed7b.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/24

# 2.1) Navigating into defog folder of the training folder

In [3]:
cd ../input 


/kaggle/input


In [4]:
cd tlvmc-parkinsons-freezing-gait-prediction

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction


In [5]:
cd train/defog

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog


In [6]:
ls #shows all csv(s) in the folder

02ea782681.csv  32d03020a9.csv  68e7e02a47.csv  9f2b3555c8.csv  daa4d27db4.csv
06414383cf.csv  38befe9f39.csv  6a20935af5.csv  a057215d88.csv  dd0ddc0bb5.csv
092b4c1819.csv  3e6987cb2d.csv  6c300fcdd5.csv  a2f1a8ab76.csv  e069a57511.csv
0c55be4384.csv  3f3b08f78d.csv  6dc94db321.csv  a8c5a489bd.csv  e1f92471b9.csv
0d7ab3a9f9.csv  3f970065e5.csv  7030643376.csv  aafcbecb5a.csv  e642d9ea5f.csv
0eaac04f17.csv  41bc215f97.csv  771d69d829.csv  afaec39b8c.csv  e971ae2a09.csv
0ec76d2d8e.csv  4520cf1068.csv  77d7d95074.csv  afc4b9d2a8.csv  edd79a7494.csv
139f60d29b.csv  4c0aec9035.csv  7a467da4f3.csv  b3891f42d0.csv  f0b8335d50.csv
13a4fe5159.csv  4c3aa8ea6e.csv  7a993fbed7.csv  b4365bba9d.csv  f17eacf7d8.csv
15508c7f41.csv  4ec23c3d98.csv  7b5346a3f2.csv  bdcff4be3a.csv  f2ffe15369.csv
18e7abc37e.csv  4f613ccf88.csv  8282009100.csv  bdda73c9be.csv  f3a921edee.csv
1d99c2eecf.csv  509a9830a6.csv  850748a138.csv  be15a0d6c5.csv  f40e8c6ebe.csv
1ff78d55e9.csv  519f292daa.csv  88c6d288fb.csv  be9d

# 2.2)importing all the necessary libraries

In [7]:
import pandas as pd
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch 
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 2.3) Merging all the csv(s) in the defog folder

In [8]:
csv_files = glob.glob('*.csv') # traverse the defog folder
dfmain = []
for file in csv_files:
    df = pd.read_csv(file)  # read the individual csv
    dfmain.append(df)        # append the df dataframe
dfmaster = pd.concat(dfmain,axis=0,ignore_index=True)
print(dfmaster) #print the merged dataframe

            Time      AccV     AccML     AccAP  StartHesitation  Turn  \
0              0 -1.002697  0.022371  0.068304                0     0   
1              1 -1.002641  0.019173  0.066162                0     0   
2              2 -0.999820  0.019142  0.067536                0     0   
3              3 -0.998023  0.018378  0.068409                0     0   
4              4 -0.998359  0.016726  0.066448                0     0   
...          ...       ...       ...       ...              ...   ...   
13525697  109120 -0.939241  0.031564 -0.394737                0     0   
13525698  109121 -0.941096  0.031582 -0.392626                0     0   
13525699  109122 -0.940131  0.029092 -0.394385                0     0   
13525700  109123 -0.939872  0.028058 -0.398664                0     0   
13525701  109124 -0.939006  0.026628 -0.398454                0     0   

          Walking  Valid   Task  
0               0  False  False  
1               0  False  False  
2               0  Fa

# 2.4) Merging all the csv(s) in the tdcsfog folder

In [9]:
cd - 

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction


In [10]:
ls # showing all the folders in the training folder

daily_metadata.csv  sample_submission.csv  tdcsfog_metadata.csv  unlabeled/
defog_metadata.csv  subjects.csv           test/
events.csv          tasks.csv              train/


In [11]:
cd train/tdcsfog 

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog


In [12]:
ls # showing all the csv(s)

003f117e14.csv  311e2722d2.csv  6e22817d93.csv  9b9e2d20c5.csv  ca9d202068.csv
009ee11563.csv  31c7065b3d.csv  6e258d0083.csv  9c9baf9458.csv  cbc907dd0e.csv
011322847a.csv  323e69055f.csv  6e6ad18f5e.csv  9cb654d456.csv  cc90f69789.csv
01d0fe7266.csv  32685c1cb7.csv  6f4013fb72.csv  9d02d58fa7.csv  cc91b27f46.csv
024418ba39.csv  3291d59f24.csv  6fb2cba36f.csv  9d161e0027.csv  ccf39d0b9e.csv
024ba3ebd5.csv  32b4bdc304.csv  70b291ba2d.csv  9d6017c17e.csv  cd0bf2d331.csv
02e8454f57.csv  33f56a4083.csv  714dc454eb.csv  9dbe8f59ae.csv  cd23f96303.csv
02edc527c0.csv  3415b61278.csv  71655655a0.csv  9dfe62541a.csv  cd7a1efefb.csv
0330ea6680.csv  343d04c4ad.csv  71bc892dd5.csv  9e0442296b.csv  cd7bca4654.csv
0383be6b43.csv  34a03b83bb.csv  7263d0d3f4.csv  9e0b9001cf.csv  cd890bcc2b.csv
03bb0c0c46.csv  34dfee952f.csv  72ee9432a7.csv  9e340ace45.csv  cdb436af61.csv
03c9d8dbfa.csv  351b10100a.csv  7411f3df87.csv  9e34d98ca6.csv  ce1c876c1a.csv
03e7159bcc.csv  35bd42e628.csv  74239820d4.csv  9e60

In [13]:
csv_files = glob.glob('*.csv')  # traverse the defog folder
dfmains = []
for file in csv_files:
    df = pd.read_csv(file)        #read the individual csv    
    dfmains.append(df)           # append the df dataframe
dfmaster2 = pd.concat(dfmains,axis=0,ignore_index=True)
print(dfmaster2) # print the merged dataframe

         Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking
0           0 -9.665890  0.042550  0.184744                0     0        0
1           1 -9.672969  0.049217  0.184644                0     0        0
2           2 -9.670260  0.033620  0.193790                0     0        0
3           3 -9.673356  0.035159  0.184369                0     0        0
4           4 -9.671458  0.043913  0.197814                0     0        0
...       ...       ...       ...       ...              ...   ...      ...
7062667  5153 -9.915920 -0.105897 -1.123455                0     0        0
7062668  5154 -9.693752 -0.066892 -1.114903                0     0        0
7062669  5155 -9.548118 -0.098315 -1.112123                0     0        0
7062670  5156 -9.469803 -0.111004 -1.130814                0     0        0
7062671  5157 -9.566318 -0.113865 -1.103353                0     0        0

[7062672 rows x 7 columns]


In [14]:
print(dfmaster2)  # merged dataframe from tdcsfog

         Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking
0           0 -9.665890  0.042550  0.184744                0     0        0
1           1 -9.672969  0.049217  0.184644                0     0        0
2           2 -9.670260  0.033620  0.193790                0     0        0
3           3 -9.673356  0.035159  0.184369                0     0        0
4           4 -9.671458  0.043913  0.197814                0     0        0
...       ...       ...       ...       ...              ...   ...      ...
7062667  5153 -9.915920 -0.105897 -1.123455                0     0        0
7062668  5154 -9.693752 -0.066892 -1.114903                0     0        0
7062669  5155 -9.548118 -0.098315 -1.112123                0     0        0
7062670  5156 -9.469803 -0.111004 -1.130814                0     0        0
7062671  5157 -9.566318 -0.113865 -1.103353                0     0        0

[7062672 rows x 7 columns]


In [15]:
print(dfmaster)  # merged dataframe from defog

            Time      AccV     AccML     AccAP  StartHesitation  Turn  \
0              0 -1.002697  0.022371  0.068304                0     0   
1              1 -1.002641  0.019173  0.066162                0     0   
2              2 -0.999820  0.019142  0.067536                0     0   
3              3 -0.998023  0.018378  0.068409                0     0   
4              4 -0.998359  0.016726  0.066448                0     0   
...          ...       ...       ...       ...              ...   ...   
13525697  109120 -0.939241  0.031564 -0.394737                0     0   
13525698  109121 -0.941096  0.031582 -0.392626                0     0   
13525699  109122 -0.940131  0.029092 -0.394385                0     0   
13525700  109123 -0.939872  0.028058 -0.398664                0     0   
13525701  109124 -0.939006  0.026628 -0.398454                0     0   

          Walking  Valid   Task  
0               0  False  False  
1               0  False  False  
2               0  Fa

# 2.5) cleaning the dfmaster dataframe.

**As the dfmaster dataframe has annotation as true and false.<br>
True denoting that the data is correct while false indicates that the data is faulty.<br>
we only want correct data to be fed into our models.<br>
we will clean the dataframe to include only that tuples which are labeled as true<br>**

In [16]:
# taking only that rows which are labeled true
dfmaster = dfmaster[(dfmaster['Task']==1) & (dfmaster['Valid']==1)]  

In [17]:
dfmaster # main dataframe 1

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task
1000,1000,-1.015138,-0.071969,0.161124,0,0,0,True,True
1001,1001,-1.018566,-0.057850,0.144988,0,0,0,True,True
1002,1002,-1.037102,-0.054200,0.129327,0,0,0,True,True
1003,1003,-1.036416,-0.060578,0.115451,0,0,0,True,True
1004,1004,-1.012407,-0.069198,0.110121,0,0,0,True,True
...,...,...,...,...,...,...,...,...,...
13524697,108120,-0.978999,0.043111,-0.392852,0,0,0,True,True
13524698,108121,-0.949352,0.016142,-0.362455,0,0,0,True,True
13524699,108122,-0.909800,-0.007553,-0.364498,0,0,0,True,True
13524700,108123,-0.882329,-0.022470,-0.374119,0,0,0,True,True


In [18]:
dfmaster2 # main dataframe 2

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.665890,0.042550,0.184744,0,0,0
1,1,-9.672969,0.049217,0.184644,0,0,0
2,2,-9.670260,0.033620,0.193790,0,0,0
3,3,-9.673356,0.035159,0.184369,0,0,0
4,4,-9.671458,0.043913,0.197814,0,0,0
...,...,...,...,...,...,...,...
7062667,5153,-9.915920,-0.105897,-1.123455,0,0,0
7062668,5154,-9.693752,-0.066892,-1.114903,0,0,0
7062669,5155,-9.548118,-0.098315,-1.112123,0,0,0
7062670,5156,-9.469803,-0.111004,-1.130814,0,0,0


In [19]:
dfmaster_new = dfmaster.drop(columns=['Valid','Task']) # dropping the valid and task attributes

In [20]:
dfmaster_new # new dataframe

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
1000,1000,-1.015138,-0.071969,0.161124,0,0,0
1001,1001,-1.018566,-0.057850,0.144988,0,0,0
1002,1002,-1.037102,-0.054200,0.129327,0,0,0
1003,1003,-1.036416,-0.060578,0.115451,0,0,0
1004,1004,-1.012407,-0.069198,0.110121,0,0,0
...,...,...,...,...,...,...,...
13524697,108120,-0.978999,0.043111,-0.392852,0,0,0
13524698,108121,-0.949352,0.016142,-0.362455,0,0,0
13524699,108122,-0.909800,-0.007553,-0.364498,0,0,0
13524700,108123,-0.882329,-0.022470,-0.374119,0,0,0


In [21]:
dfmaster2

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.665890,0.042550,0.184744,0,0,0
1,1,-9.672969,0.049217,0.184644,0,0,0
2,2,-9.670260,0.033620,0.193790,0,0,0
3,3,-9.673356,0.035159,0.184369,0,0,0
4,4,-9.671458,0.043913,0.197814,0,0,0
...,...,...,...,...,...,...,...
7062667,5153,-9.915920,-0.105897,-1.123455,0,0,0
7062668,5154,-9.693752,-0.066892,-1.114903,0,0,0
7062669,5155,-9.548118,-0.098315,-1.112123,0,0,0
7062670,5156,-9.469803,-0.111004,-1.130814,0,0,0


In [22]:
# merging the two main dataframe
main_data = pd.concat([dfmaster2,dfmaster_new],axis=0,ignore_index=True)

In [23]:
main_data # this will be cleaned and fed into neural networks

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.665890,0.042550,0.184744,0,0,0
1,1,-9.672969,0.049217,0.184644,0,0,0
2,2,-9.670260,0.033620,0.193790,0,0,0
3,3,-9.673356,0.035159,0.184369,0,0,0
4,4,-9.671458,0.043913,0.197814,0,0,0
...,...,...,...,...,...,...,...
11153197,108120,-0.978999,0.043111,-0.392852,0,0,0
11153198,108121,-0.949352,0.016142,-0.362455,0,0,0
11153199,108122,-0.909800,-0.007553,-0.364498,0,0,0
11153200,108123,-0.882329,-0.022470,-0.374119,0,0,0


# 2.6) Normalization of features 

**As neural networks accept values between 0 and 1 and hence we normalize the time feature so that it will fit the model.<br>Normalization is achieved by dividing individual time values with length of the time attribute.**

In [24]:
# normalizing the timefeature
main_data.Time = main_data.Time/((len(main_data.Time))-1)

In [25]:
main_data # showing the final dataframe

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0.000000e+00,-9.665890,0.042550,0.184744,0,0,0
1,8.966036e-08,-9.672969,0.049217,0.184644,0,0,0
2,1.793207e-07,-9.670260,0.033620,0.193790,0,0,0
3,2.689811e-07,-9.673356,0.035159,0.184369,0,0,0
4,3.586414e-07,-9.671458,0.043913,0.197814,0,0,0
...,...,...,...,...,...,...,...
11153197,9.694078e-03,-0.978999,0.043111,-0.392852,0,0,0
11153198,9.694168e-03,-0.949352,0.016142,-0.362455,0,0,0
11153199,9.694257e-03,-0.909800,-0.007553,-0.364498,0,0,0
11153200,9.694347e-03,-0.882329,-0.022470,-0.374119,0,0,0


In [26]:
# printing individual classes
Y1 = main_data['StartHesitation'] 
Y2 = main_data['Turn']
Y3 = main_data['Walking']

In [27]:
print(Y1)

0           0
1           0
2           0
3           0
4           0
           ..
11153197    0
11153198    0
11153199    0
11153200    0
11153201    0
Name: StartHesitation, Length: 11153202, dtype: int64


In [28]:
print(Y2)

0           0
1           0
2           0
3           0
4           0
           ..
11153197    0
11153198    0
11153199    0
11153200    0
11153201    0
Name: Turn, Length: 11153202, dtype: int64


In [29]:
print(Y3)

0           0
1           0
2           0
3           0
4           0
           ..
11153197    0
11153198    0
11153199    0
11153200    0
11153201    0
Name: Walking, Length: 11153202, dtype: int64


# 2.7) Unbalanced classes in dataframe

**By printing the above classes we clearly see that the dataset is unbalanced by a huge margin.<br>
Parkinson freezing of gait is a very rare event and thatswhy the dataset looks like this.<br>
This unbalanced dataframe cannot be fed into neural networks.<br> 
The neural network will always predict 0.<br>**


# 2.8) Balancing the dataframe

In [30]:
#finding the index of starthesitation class where it is zero and one.
#using numpy arrays
y1_zeroes = np.where(Y1 == 0)[0] 
y1_ones = np.where(Y1 == 1)[0]

In [31]:
y1_zeroes # printing the index array

array([       0,        1,        2, ..., 11153199, 11153200, 11153201])

In [32]:
y1_ones #printing the index array

array([  108017,   108018,   108019, ..., 11082473, 11082474, 11082475])

In [33]:
# balancing the dataset
num1_ones = (Y1 == 1).sum()
np.random.seed(45)
y1_zeroes = np.random.choice(np.where(Y1==0)[0],size=num1_ones,replace=False)
y1_balanced_index = np.sort(np.concatenate([y1_zeroes,y1_ones]))

In [34]:
# start hesitation dataframe is balanced
Balance_maindata = main_data.iloc[y1_balanced_index,:]
Balance_maindata 

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
48,0.000004,-9.680708,0.020887,0.187325,0,0,0
49,0.000004,-9.676230,0.023066,0.182803,0,0,0
105,0.000009,-9.675154,0.033351,0.159722,0,0,0
184,0.000016,-9.675474,0.039896,0.170911,0,0,0
194,0.000017,-9.681351,0.022568,0.187034,0,0,0
...,...,...,...,...,...,...,...
11153040,0.009680,-0.921942,-0.030008,-0.485369,0,0,0
11153117,0.009687,-0.946454,0.030874,-0.372546,0,0,0
11153119,0.009687,-0.944098,0.016962,-0.371695,0,0,0
11153128,0.009688,-0.945338,0.031899,-0.370115,0,0,0


In [35]:
# checking the length of tuples with starthesitation as 0
print(Balance_maindata['StartHesitation'].value_counts()[0])


305290


In [36]:
# checking the length of tuples with starthesitation as 1
print(Balance_maindata['StartHesitation'].value_counts()[1])

305290


In [37]:
# dropping the turn and walking attribute
balanced_data_for_starthesitation  = Balance_maindata.drop(columns=['Turn','Walking'])

In [38]:
balanced_data_for_starthesitation

,Time,AccV,AccML,AccAP,StartHesitation
48,0.000004,-9.680708,0.020887,0.187325,0
49,0.000004,-9.676230,0.023066,0.182803,0
105,0.000009,-9.675154,0.033351,0.159722,0
184,0.000016,-9.675474,0.039896,0.170911,0
194,0.000017,-9.681351,0.022568,0.187034,0
...,...,...,...,...,...
11153040,0.009680,-0.921942,-0.030008,-0.485369,0
11153117,0.009687,-0.946454,0.030874,-0.372546,0
11153119,0.009687,-0.944098,0.016962,-0.371695,0
11153128,0.009688,-0.945338,0.031899,-0.370115,0


***Now we have one balanced dataframe for the model that will predict the starthesitation event***

In [39]:
# doing with y2
y2_zeroes = np.where(Y2 == 0)[0]
y2_ones = np.where(Y2 == 1)[0]

In [40]:
y2_zeroes # printing the index array

array([       0,        1,        2, ..., 11153199, 11153200, 11153201])

In [41]:
y2_ones # printing the index array

array([    1750,     1751,     1752, ..., 11146509, 11146510, 11146511])

In [42]:
# balancing the dataset
num1_ones = (Y2 == 1).sum()
np.random.seed(45)
y2_zeroes = np.random.choice(np.where(Y2==0)[0],size=num1_ones,replace=False)
y2_balanced_index = np.sort(np.concatenate([y2_zeroes,y2_ones]))

In [43]:
# Balance_maindata2 dataframe is balanced
Balance_maindata2 = main_data.iloc[y2_balanced_index,:]
Balance_maindata2 # only turn is balanced

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
4,3.586414e-07,-9.671458,0.043913,0.197814,0,0,0
8,7.172829e-07,-9.664340,0.035000,0.195657,0,0,0
19,1.703547e-06,-9.682555,0.046558,0.175329,0,0,0
20,1.793207e-06,-9.680701,0.046148,0.170617,0,0,0
24,2.151849e-06,-9.682190,0.035471,0.207058,0,0,0
...,...,...,...,...,...,...,...
11153189,9.693361e-03,-0.930295,0.126166,-0.316116,0,0,0
11153190,9.693450e-03,-0.930729,0.115029,-0.320000,0,0,0
11153195,9.693899e-03,-0.979825,0.063121,-0.435634,0,0,0
11153196,9.693988e-03,-0.989724,0.059319,-0.429806,0,0,0


In [44]:
# checking if the dataframe is balanced
print(Balance_maindata2['Turn'].value_counts()[0])

2265611


In [45]:
# checking if the dataframe is balanced
print(Balance_maindata2['Turn'].value_counts()[1])

2265611


In [46]:
# dropping the starthesitation and walking attribute
Balance_maindata2 = Balance_maindata2.drop(columns=['StartHesitation','Walking'])

In [47]:
Balance_maindata2

,Time,AccV,AccML,AccAP,Turn
4,3.586414e-07,-9.671458,0.043913,0.197814,0
8,7.172829e-07,-9.664340,0.035000,0.195657,0
19,1.703547e-06,-9.682555,0.046558,0.175329,0
20,1.793207e-06,-9.680701,0.046148,0.170617,0
24,2.151849e-06,-9.682190,0.035471,0.207058,0
...,...,...,...,...,...
11153189,9.693361e-03,-0.930295,0.126166,-0.316116,0
11153190,9.693450e-03,-0.930729,0.115029,-0.320000,0
11153195,9.693899e-03,-0.979825,0.063121,-0.435634,0
11153196,9.693988e-03,-0.989724,0.059319,-0.429806,0


In [48]:
# checking if the dataframe is balanced
print(Balance_maindata2['Turn'].value_counts()[0])

2265611


In [49]:
# checking if the dataframe is balanced
print(Balance_maindata2['Turn'].value_counts()[1])

2265611


***Now we have one balanced dataframe for the model that will predict the turn event***

In [50]:
# doing for y3 for third model
y3_zeroes = np.where(Y3 == 0)[0]
y3_ones = np.where(Y3 == 1)[0]

In [51]:
y3_zeroes # printing the index array

array([       0,        1,        2, ..., 11153199, 11153200, 11153201])

In [52]:
y3_ones # printing the index array

array([  112200,   112201,   112202, ..., 11071418, 11071419, 11071420])

In [53]:
# balancing the dataset
num1_ones = (Y3 == 1).sum()
np.random.seed(45)
y3_zeroes = np.random.choice(np.where(Y3==0)[0],size=num1_ones,replace=False)
y3_balanced_index = np.sort(np.concatenate([y3_zeroes,y3_ones]))

In [54]:
# Balance_maindata3 dataframe is balanced
Balance_maindata3 = main_data.iloc[y3_balanced_index,:]
Balance_maindata3 

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
48,0.000004,-9.680708,0.020887,0.187325,0,0,0
49,0.000004,-9.676230,0.023066,0.182803,0,0,0
105,0.000009,-9.675154,0.033351,0.159722,0,0,0
131,0.000012,-9.668657,0.037558,0.150582,0,0,0
184,0.000016,-9.675474,0.039896,0.170911,0,0,0
...,...,...,...,...,...,...,...
11153082,0.009684,-0.926235,0.064119,-0.384614,0,0,0
11153107,0.009686,-0.928599,0.022964,-0.374787,0,0,0
11153119,0.009687,-0.944098,0.016962,-0.371695,0,0,0
11153128,0.009688,-0.945338,0.031899,-0.370115,0,0,0


In [55]:
# dropping the turn and starthesitationa attribute
Balance_maindata3 = Balance_maindata3.drop(columns=['StartHesitation','Turn'])

In [56]:
Balance_maindata3

,Time,AccV,AccML,AccAP,Walking
48,0.000004,-9.680708,0.020887,0.187325,0
49,0.000004,-9.676230,0.023066,0.182803,0
105,0.000009,-9.675154,0.033351,0.159722,0
131,0.000012,-9.668657,0.037558,0.150582,0
184,0.000016,-9.675474,0.039896,0.170911,0
...,...,...,...,...,...
11153082,0.009684,-0.926235,0.064119,-0.384614,0
11153107,0.009686,-0.928599,0.022964,-0.374787,0
11153119,0.009687,-0.944098,0.016962,-0.371695,0
11153128,0.009688,-0.945338,0.031899,-0.370115,0


In [57]:
# checking if the dataframe is balanced
print(Balance_maindata3['Walking'].value_counts()[0])

306356


In [58]:
# checking if the dataframe is balanced
print(Balance_maindata3['Walking'].value_counts()[1])

306356


***Now we have one balanced dataframe for the model that will predict the walking event***


# 2.8)Now we have three balanced dateframe for three models

In [59]:
balanced_data_for_starthesitation # for first model

,Time,AccV,AccML,AccAP,StartHesitation
48,0.000004,-9.680708,0.020887,0.187325,0
49,0.000004,-9.676230,0.023066,0.182803,0
105,0.000009,-9.675154,0.033351,0.159722,0
184,0.000016,-9.675474,0.039896,0.170911,0
194,0.000017,-9.681351,0.022568,0.187034,0
...,...,...,...,...,...
11153040,0.009680,-0.921942,-0.030008,-0.485369,0
11153117,0.009687,-0.946454,0.030874,-0.372546,0
11153119,0.009687,-0.944098,0.016962,-0.371695,0
11153128,0.009688,-0.945338,0.031899,-0.370115,0


In [60]:
Balance_maindata2 #for second model

,Time,AccV,AccML,AccAP,Turn
4,3.586414e-07,-9.671458,0.043913,0.197814,0
8,7.172829e-07,-9.664340,0.035000,0.195657,0
19,1.703547e-06,-9.682555,0.046558,0.175329,0
20,1.793207e-06,-9.680701,0.046148,0.170617,0
24,2.151849e-06,-9.682190,0.035471,0.207058,0
...,...,...,...,...,...
11153189,9.693361e-03,-0.930295,0.126166,-0.316116,0
11153190,9.693450e-03,-0.930729,0.115029,-0.320000,0
11153195,9.693899e-03,-0.979825,0.063121,-0.435634,0
11153196,9.693988e-03,-0.989724,0.059319,-0.429806,0


In [61]:
Balance_maindata3 # for third modelhave

,Time,AccV,AccML,AccAP,Walking
48,0.000004,-9.680708,0.020887,0.187325,0
49,0.000004,-9.676230,0.023066,0.182803,0
105,0.000009,-9.675154,0.033351,0.159722,0
131,0.000012,-9.668657,0.037558,0.150582,0
184,0.000016,-9.675474,0.039896,0.170911,0
...,...,...,...,...,...
11153082,0.009684,-0.926235,0.064119,-0.384614,0
11153107,0.009686,-0.928599,0.022964,-0.374787,0
11153119,0.009687,-0.944098,0.016962,-0.371695,0
11153128,0.009688,-0.945338,0.031899,-0.370115,0


# 2.9) Neural Networks

**TRAINING AND INFERENCE**

In [62]:
# Split dataframe into features and labels
#model1
features_model1 = balanced_data_for_starthesitation.iloc[:, :4].values
labels_model1 = balanced_data_for_starthesitation.iloc[:, 4].values

In [92]:
# Split dataset into training and test sets
train_features, test_features, train_labels, test_labels = train_test_split(features_model1, labels_model1, test_size=0.2)

# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __getitem__(self, index):
        return (torch.Tensor(self.features[index])).to(device), (torch.Tensor([self.labels[index]])).to(device)
    
    def __len__(self):
        return len(self.labels)

# Create PyTorch dataloaders for training and test sets for model1
train_dataset_model1 = CustomDataset(train_features, train_labels)
train_dataloader_model1 = DataLoader(train_dataset_model1, batch_size=10000, shuffle=True)

test_dataset_model1 = CustomDataset(test_features, test_labels)
test_dataloader_model1 = DataLoader(test_dataset_model1, batch_size=122116, shuffle=False)


In [65]:
# setting up the gpu for training
device='cuda'

In [66]:
# checking the training dataloader for model1
for batch_idx, (data, target) in enumerate(train_dataloader_model1):
    # Print the shape of the data and target tensors
    print(f"Batch {batch_idx+1} - Data shape: {data.shape}, Target shape: {target.shape}")
    # Print the first few rows of data and target tensors
    print(f"features: {data[:5]}\nlabels: {target[:5]}")
    # Exit the loop after the first batch
    break

Batch 1 - Data shape: torch.Size([1000, 4]), Target shape: torch.Size([1000, 1])
features: tensor([[ 3.7137e-03, -9.3766e+00,  1.0724e+00,  4.7580e+00],
        [ 3.5766e-04, -1.0592e+01, -4.4957e+00,  4.6984e+00],
        [ 3.8491e-04, -1.1321e+00, -3.6361e-03, -1.9272e-01],
        [ 9.6295e-05, -9.6233e+00,  3.3206e-01,  1.4732e+00],
        [ 4.3243e-04, -9.6308e+00,  1.1770e+00,  1.5102e+00]], device='cuda:0')
labels: tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]], device='cuda:0')


In [67]:
# checking the test dataloader for model1
for batch_idx, (data, target) in enumerate(test_dataloader_model1):
    # Print the shape of the data and target tensors
    print(f"Batch {batch_idx+1} - Data shape: {data.shape}, Target shape: {target.shape}")
    # Print the first few rows of data and target tensors
    print(f"features: {data[:5]}\nlabels: {target[:5]}")
    # Exit the loop after the first batch
    break

Batch 1 - Data shape: torch.Size([1000, 4]), Target shape: torch.Size([1000, 1])
features: tensor([[ 3.3990e-04, -1.0232e+01,  1.3430e+00,  2.6897e+00],
        [ 1.3099e-04, -8.7378e+00,  9.0547e-01,  3.8164e+00],
        [ 9.0028e-04, -8.6337e+00, -4.0376e-01,  3.9414e+00],
        [ 1.6847e-04, -9.0221e+00, -1.3012e+00,  1.2756e+00],
        [ 2.2639e-04, -9.6540e+00, -1.7264e+00,  2.5490e+00]], device='cuda:0')
labels: tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')


In [68]:
# Split dataframe into features and labels
# model2
features_model2 = Balance_maindata2.iloc[:, :4].values
labels_model2 = Balance_maindata2.iloc[:, 4].values

In [69]:
# Create PyTorch dataloaders for training and test sets for model2
train_features, test_features, train_labels, test_labels = train_test_split(features_model2, labels_model2, test_size=0.2)
train_dataset_model2 = CustomDataset(train_features, train_labels)
train_dataloader_model2 = DataLoader(train_dataset_model2,batch_size=100, shuffle=True)

test_dataset_model2 = CustomDataset(test_features, test_labels)
test_dataloader_model2 = DataLoader(test_dataset_model2,batch_size=100, shuffle=False)

In [70]:
# checking the training dataloader for model2
for batch_idx, (data, target) in enumerate(train_dataloader_model2):
    # Print the shape of the data and target tensors
    print(f"Batch {batch_idx+1} - Data shape: {data.shape}, Target shape: {target.shape}")
    # Print the first few rows of data and target tensors
    print(f"features: {data[:5]}\nlabels: {target[:5]}")
    # Exit the loop after the first batch
    break

Batch 1 - Data shape: torch.Size([100, 4]), Target shape: torch.Size([100, 1])
features: tensor([[ 1.3655e-04, -7.9286e+00, -1.1654e+00,  2.4941e+00],
        [ 2.5741e-04, -9.2913e+00, -2.1740e-01,  2.6743e+00],
        [ 9.9209e-04, -9.8431e+00, -5.3907e-01,  2.6318e+00],
        [ 7.0392e-04, -9.0162e+00,  9.8079e-01,  2.1259e+00],
        [ 8.2560e-03, -9.8438e-01,  6.7026e-02, -4.1466e-01]], device='cuda:0')
labels: tensor([[0.],
        [1.],
        [1.],
        [0.],
        [0.]], device='cuda:0')


In [71]:
# checking the testing dataloader for model1
for batch_idx, (data, target) in enumerate(train_dataloader_model2):
    # Print the shape of the data and target tensors
    print(f"Batch {batch_idx+1} - Data shape: {data.shape}, Target shape: {target.shape}")
    # Print the first few rows of data and target tensors
    print(f"features: {data[:5]}\nlabels: {target[:5]}")
    # Exit the loop after the first batch
    break

Batch 1 - Data shape: torch.Size([100, 4]), Target shape: torch.Size([100, 1])
features: tensor([[ 9.0933e-03, -9.0625e-01, -3.1250e-02, -3.7500e-01],
        [ 5.2742e-03, -8.8907e+00, -7.4618e-01,  4.6539e+00],
        [ 5.3707e-03, -8.9207e+00, -1.1920e+00,  3.9904e+00],
        [ 5.5048e-03, -9.9228e-01,  1.4947e-01, -1.9639e-01],
        [ 9.9261e-03, -9.1205e-01, -3.3572e-01, -3.3291e-01]], device='cuda:0')
labels: tensor([[1.],
        [0.],
        [1.],
        [1.],
        [0.]], device='cuda:0')


In [72]:
# Split dataframe into features and labels
# model3
features_model3 = Balance_maindata3.iloc[:, :4].values
labels_model3 = Balance_maindata3.iloc[:, 4].values

In [73]:
# Create PyTorch dataloaders for training and test sets for model3
train_features, test_features, train_labels, test_labels = train_test_split(features_model3, labels_model3, test_size=0.2)
train_dataset_model3 = CustomDataset(train_features, train_labels)
train_dataloader_model3 = DataLoader(train_dataset_model3,batch_size=100, shuffle=True)

test_dataset_model3 = CustomDataset(test_features, test_labels)
test_dataloader_model3 = DataLoader(test_dataset_model3,batch_size=100, shuffle=False)

In [74]:
# checking the training dataloader for model3
for batch_idx, (data, target) in enumerate(train_dataloader_model3):
    # Print the shape of the data and target tensors
    print(f"Batch {batch_idx+1} - Data shape: {data.shape}, Target shape: {target.shape}")
    # Print the first few rows of data and target tensors
    print(f"features: {data[:5]}\nlabels: {target[:5]}")
    # Exit the loop after the first batch
    break

Batch 1 - Data shape: torch.Size([100, 4]), Target shape: torch.Size([100, 1])
features: tensor([[ 5.5912e-04, -9.1023e+00, -1.2328e+00,  2.1856e-01],
        [ 8.8942e-03, -9.7072e-01, -3.0813e-02, -1.7504e-01],
        [ 5.1835e-03, -8.2599e+00, -4.0346e+00,  3.3215e+00],
        [ 2.8754e-03, -8.8174e+00, -1.9612e+00,  3.7374e+00],
        [ 2.0801e-04, -9.9754e+00, -3.0370e-01, -6.3806e-01]], device='cuda:0')
labels: tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.]], device='cuda:0')


In [75]:
# checking the testing dataloader for model3
for batch_idx, (data, target) in enumerate(test_dataloader_model3):
    # Print the shape of the data and target tensors
    print(f"Batch {batch_idx+1} - Data shape: {data.shape}, Target shape: {target.shape}")
    # Print the first few rows of data and target tensors
    print(f"features: {data[:5]}\nlabels: {target[:5]}")
    # Exit the loop after the first batch
    break

Batch 1 - Data shape: torch.Size([100, 4]), Target shape: torch.Size([100, 1])
features: tensor([[ 1.4229e-02, -1.0127e+00, -1.0283e-01, -2.4680e-01],
        [ 1.0292e-02, -8.0019e-01,  1.5433e-01, -4.5918e-01],
        [ 3.1076e-04, -7.9716e+00, -4.1251e-01,  4.6633e+00],
        [ 7.1810e-03, -8.1913e+00, -2.0724e+00,  3.2766e+00],
        [ 1.5691e-05, -9.6457e+00, -1.5314e-01, -5.7920e-01]], device='cuda:0')
labels: tensor([[1.],
        [0.],
        [0.],
        [1.],
        [0.]], device='cuda:0')


In [76]:
# defining the neural network
# This is a pyTorch framework
# The model faetures linear as well as non linear relu layers
# There are two hidden layers
torch.manual_seed(90)
class neuralnetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(4,5)
    self.layer2 = nn.Linear(5,5)  # layers 
    self.layer3 = nn.Linear(5,5)
    self.layer4 = nn.Linear(5,5)
    self.layer5 = nn.Linear(5,5)
    self.layer6 = nn.Linear(5,5)
    self.layer7 = nn.Linear(5,5)
    self.layer8 = nn.Linear(5,5)
    self.layer9 = nn.Linear(5,5)
    self.layer10 = nn.Linear(5,1)
    self.relu = nn.ReLU()
  def forward(self,x): # forward method
    return self.layer10(self.relu(self.layer9(self.relu(self.layer8(self.relu(self.layer7(self.relu(self.layer6(self.relu(self.layer5(self.relu(self.layer4(self.relu(self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))))))))))))))))

In [77]:
# each model is the instance of the neuralnetwork class
model1 = neuralnetwork()
model2 = neuralnetwork()
model3 = neuralnetwork()
# declaring the loss function and optimizer for training
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params = model1.parameters(),lr=0.001) #learning rate

In [78]:
# printing the model details and parameters
print(model1.state_dict)
print(model2.state_dict)
print(model3.state_dict)

<bound method Module.state_dict of neuralnetwork(
  (layer1): Linear(in_features=4, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=5, bias=True)
  (layer3): Linear(in_features=5, out_features=5, bias=True)
  (layer4): Linear(in_features=5, out_features=5, bias=True)
  (layer5): Linear(in_features=5, out_features=5, bias=True)
  (layer6): Linear(in_features=5, out_features=5, bias=True)
  (layer7): Linear(in_features=5, out_features=5, bias=True)
  (layer8): Linear(in_features=5, out_features=5, bias=True)
  (layer9): Linear(in_features=5, out_features=5, bias=True)
  (layer10): Linear(in_features=5, out_features=1, bias=True)
  (relu): ReLU()
)>
<bound method Module.state_dict of neuralnetwork(
  (layer1): Linear(in_features=4, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=5, bias=True)
  (layer3): Linear(in_features=5, out_features=5, bias=True)
  (layer4): Linear(in_features=5, out_features=5, bias=True)
  (layer5): Linear(in_fea

In [79]:
pip install tqdm #importing tqdm library to visualise the progress

Note: you may need to restart the kernel to use updated packages.


In [91]:
len(test_features)

122116

In [93]:
# training the first model
from tqdm import tqdm
device='cuda'
model1=model1.to(device)
train_dataloader_model1=train_dataloader_model1
epochs =500
train_loss = []  #list for storing values
count_epochs = []
testloss = []
for epoch in tqdm(range(epochs)):
    model1.train()
    for features,labels in train_dataloader_model1:
       ylogits = model1(features)  
       ypred = torch.round(torch.sigmoid(ylogits))
       loss  = loss_fn(ylogits,labels) 
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
       model1.eval()
    with torch.inference_mode():
      for test_features,test_labels in test_dataloader_model1:
        logits = model1(test_features)
        resultstest = torch.round(torch.sigmoid(logits))
        test_loss = loss_fn(resultstest,test_labels)
    print(f"epochs:{epoch} | trainingloss:{loss} | testloss:{test_loss}")
    train_loss.append(loss) # appending the lists
    testloss.append(test_loss)
    count_epochs.append(epoch)

  0%|          | 1/500 [00:36<5:07:29, 36.97s/it]

epochs:0 | trainingloss:0.6946214437484741 | testloss:0.6943233609199524


  0%|          | 2/500 [01:13<5:06:07, 36.88s/it]

epochs:1 | trainingloss:0.6937646865844727 | testloss:0.6942898035049438


  1%|          | 3/500 [01:50<5:06:04, 36.95s/it]

...

epochs:5 | trainingloss:0.6950648427009583 | testloss:0.6941660046577454


  1%|▏         | 7/500 [04:19<5:04:39, 37.08s/it]

epochs:6 | trainingloss:0.6944117546081543 | testloss:0.6941370368003845


  1%|▏         | 7/500 [04:41<5:29:53, 40.15s/it]


KeyboardInterrupt: 

In [ ]:
# checking the loss by plotting epochs vs training loss
import matplotlib.pyplot as plt
plt.title("MODEL1")
plt.plot(count_epochs,np.array(torch.tensor(train_loss)))
plt.xlabel("epochs")
plt.ylabel("training loss")
plt.show()

In [ ]:
# finally predicting on test dataset for starthesitation event in the form of tensors
with torch.inference_mode():
    for items in range(1):
     for features,labels in test_dataloader_model1:
        features,labels=features.to(device),labels.to(device)
        logits = model1(features)
        results = torch.round(torch.sigmoid(logits))
        print(results)   # for start hesitation the model predicts

In [ ]:
model1.state_dict()

In [ ]:
# training the second model
from tqdm import tqdm
epochs =100
train_loss = []
count_epochs = []
model2=model2.to(device)
for epoch in tqdm(range(epochs)):
    model2.train()
    for features,labels in train_dataloader_model2:
       features,labels=features.to(device),labels.to(device) 
       ylogits = model2(features)  
       ypred = torch.round(torch.sigmoid(ylogits))
       loss  = loss_fn(ylogits,labels) 
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
       model2.eval()
    print(f"epochs:{epoch} | trainingloss:{loss}")
    train_loss.append(loss)
    count_epochs.append(epoch)

In [ ]:
# checking the loss by plotting epochs vs training loss
import matplotlib.pyplot as plt
plt.title("MODEL2")
plt.plot(count_epochs,np.array(torch.tensor(train_loss)))
plt.xlabel("epochs")
plt.ylabel("training loss")
plt.show()

In [ ]:
# training the third model
epochs =100
train_loss = []
count_epochs = []
model3=model3.to(device)
for epoch in tqdm(range(epochs)):
    model3.train()
    for features,labels in train_dataloader_model3:
       features,labels=features.to(device),labels.to(device)  
       ylogits = model3(features)  
       ypred = torch.round(torch.sigmoid(ylogits))
       loss  = loss_fn(ylogits,labels) 
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
       model3.eval()
    print(f"epochs:{epoch} | trainingloss:{loss}")
    train_loss.append(loss)
    count_epochs.append(epoch)

In [ ]:
# checking the loss by plotting epochs vs training loss
import matplotlib.pyplot as plt
plt.title("MODEL3")
plt.plot(count_epochs,np.array(torch.tensor(train_loss)))
plt.xlabel("epochs")
plt.ylabel("training loss")
plt.show()